In [19]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers

# Assignment 07: Autoencoders

We will look here at autoencoders which is a combination of two neural networks: one for compressing an input to lower dimensional space and one for upsampling again to the original size of the input. In the best case, this acts as an identity map. 

## Exercise 07.1: Image compression

In this exercise we will look at image compression and decompression, for which a convolutional neural network works best. First, load the MNIST data set. Every image therein consists of 28 x 28 pixels, meaning that the input space is $784$-dimensional. Thus, the decompressing should happen to a space of dimension $d < 784$. 

In [59]:
(x_train_unnormalized, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train_unnormalized / 255, -1)

In [84]:
input_img = keras.Input(shape=(28, 28, 1))
input_encoder = (28, 28, 1)

encoder_model = keras.Sequential(

    [
        keras.Input(shape=input_encoder),
        layers.Conv2D(32, kernel_size=(2, 2), activation="relu", padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2), padding='same'),        
        layers.Conv2D(32, kernel_size=(2, 2), activation="relu", padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
        layers.Conv2D(64, kernel_size=(2,2), activation='relu', padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    ]
)

encoder_model.summary()

input_decoder = (4, 4, 64)

decoder_model = keras.Sequential(

    [
        keras.Input(shape=input_decoder),
        layers.Conv2D(64, kernel_size=(2, 2), activation="relu", padding='same'),
        layers.UpSampling2D(size=(2, 2)),        
        layers.Conv2D(32, kernel_size=(2, 2), activation="relu"),
        layers.UpSampling2D(size=(2,2)),
        layers.Conv2D(32, kernel_size=(2,2), activation='relu', padding='same'),
        layers.UpSampling2D(size=(2,2)),
        layers.Conv2D(1, (2, 2), activation='sigmoid', padding='same')
    ]
)

decoder_model.summary()

Model: "sequential_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_117 (Conv2D)             │ (None, 28, 28, 32)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_75 (MaxPooling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_118 (Conv2D)             │ (None, 14, 14, 32)     │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_76 (MaxPooling2D) │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_119 (Conv2D)             │ (None, 7, 7, 64)       │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_77 (MaxPooling2D) │ (None, 4, 4, 64)       │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,544 (49.00 KB)

 Trainable params: 12,544 (49.00 KB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_120 (Conv2D)             │ (None, 4, 4, 64)       │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_40 (UpSampling2D) │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_121 (Conv2D)             │ (None, 7, 7, 32)       │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_41 (UpSampling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_122 (Conv2D)             │ (None, 14, 14, 32)     │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_42 (UpSampling2D) │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_123 (Conv2D)             │ (None, 28, 28, 1)      │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,929 (113.00 KB)

 Trainable params: 28,929 (113.00 KB)

 Non-trainable params: 0 (0.00 B)

In [88]:
autencoder = keras.Model(input_img, decoder_model(encoder_model(input_img)))

In [90]:
epochs = 20
batch_size = 128

autencoder.compile(optimizer='adam', loss='binary_crossentropy')
autencoder.fit(x_train, x_train, epochs=epochs, batch_size=batch_size)

Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - loss: 0.1442
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - loss: 0.1162
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - loss: 0.1100
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - loss: 0.1046
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 0.1013
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 0.0983
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 0.0964
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 0.0944
Epoch 9/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 0.0932
Epoch 10/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 0.0919
Epoch 11/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 0.0908
Epoch 12/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 0.0901
Epoch 13/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - loss: 0.0892
Epoch 14/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 25ms/step - loss: 0.0884
Epoch 15/20
469/469 ━━━━━━━━━